# 서울시 지하철 시간대별 유동인구 분석 및 상위역 출력 (일평균)

- 출근/퇴근 시간대별 승하차 히트맵 시각화
- 상위 10개 혼잡역 (일일 평균 인원 기준) 콘솔에 출력

In [1]:
import pandas as pd

# 데이터 로드
df_time = pd.read_csv('seoul_subway_by_time.csv', encoding='cp949')
df_xy = pd.read_csv('seoul_subway_xy.csv')

# 필요한 컬럼
cols_needed = ['지하철역', '08시-09시 승차인원', '08시-09시 하차인원', '18시-19시 승차인원', '18시-19시 하차인원']
df_time_filtered = df_time[cols_needed].copy()
df_time_filtered['지하철역'] = df_time_filtered['지하철역'].str.strip()

# 집계
df_grouped = df_time_filtered.groupby('지하철역').sum().reset_index()

# 좌표 전처리 및 병합
df_xy['역한글명칭'] = df_xy['역한글명칭'].str.strip()
df_coords = df_xy.groupby('역한글명칭')[['환승역X좌표', '환승역Y좌표']].mean().reset_index()

df = pd.merge(df_grouped, df_coords, left_on='지하철역', right_on='역한글명칭', how='inner')

In [3]:
# 일일 평균 계산 및 상위 10개 출력 함수 정의
columns_to_avg = [
    '08시-09시 승차인원', '08시-09시 하차인원',
    '18시-19시 승차인원', '18시-19시 하차인원'
]

df_avg = df.copy()
for col in columns_to_avg:
    df_avg[col] = df_avg[col] / 31

def print_top10(df, col, title):
    top10 = df.sort_values(by=col, ascending=False).head(10).reset_index(drop=True)
    print(f"\n📌 {title} 상위 10개 역 (일평균 기준)")
    for i, row in top10.iterrows():
        station = row['지하철역']
        count = int(round(row[col]))
        print(f"{i+1}. {station} - {count:,}명")

In [5]:
# 출력 실행
print_top10(df_avg, '08시-09시 승차인원', '출근시간 승차')
print_top10(df_avg, '08시-09시 하차인원', '출근시간 하차')
print_top10(df_avg, '18시-19시 승차인원', '퇴근시간 승차')
print_top10(df_avg, '18시-19시 하차인원', '퇴근시간 하차')


📌 출근시간 승차 상위 10개 역 (일평균 기준)
1. 신림 - 1,259,489명
2. 사당 - 781,158명
3. 구로디지털단지 - 759,678명
4. 연신내 - 657,901명
5. 신도림 - 652,767명
6. 까치산 - 624,030명
7. 잠실(송파구청) - 620,633명
8. 서울대입구(관악구청) - 607,001명
9. 화곡 - 592,249명
10. 수유(강북구청) - 574,293명

📌 출근시간 하차 상위 10개 역 (일평균 기준)
1. 가산디지털단지 - 2,182,671명
2. 선릉 - 1,681,695명
3. 시청 - 1,561,689명
4. 서울역 - 1,488,051명
5. 여의도 - 1,429,162명
6. 역삼 - 1,393,149명
7. 강남 - 1,319,144명
8. 을지로입구 - 1,277,415명
9. 삼성(무역센터) - 1,085,319명
10. 종각 - 1,017,228명

📌 퇴근시간 승차 상위 10개 역 (일평균 기준)
1. 가산디지털단지 - 1,663,505명
2. 선릉 - 1,421,336명
3. 서울역 - 1,420,504명
4. 강남 - 1,397,337명
5. 시청 - 1,256,622명
6. 여의도 - 1,233,526명
7. 역삼 - 1,112,520명
8. 을지로입구 - 1,082,443명
9. 잠실(송파구청) - 1,019,663명
10. 삼성(무역센터) - 994,653명

📌 퇴근시간 하차 상위 10개 역 (일평균 기준)
1. 홍대입구 - 1,106,624명
2. 사당 - 1,058,363명
3. 신림 - 971,702명
4. 잠실(송파구청) - 911,782명
5. 강남 - 885,566명
6. 고속터미널 - 876,876명
7. 건대입구 - 857,142명
8. 신도림 - 771,750명
9. 서울역 - 689,378명
10. 구로디지털단지 - 656,293명
